# OGGM glacier simulator

App to initialize a simple glacier and look how it develops under changing mass balance profiles.

## Import packages

import plotting libaries

In [ ]:
import holoviews as hv
from holoviews import opts
from holoviews.streams import Stream, param
import panel as pn

# is needed for linecolor by velocity/thickness colorplots
from bokeh.transform import linear_cmap

hv.extension('bokeh', width=100)
pn.extension()

import OGGM packages

In [ ]:
# Constants for initialization
from functools import partial
from oggm.core.flowline import FluxBasedModel, RectangularBedFlowline, TrapezoidalBedFlowline, ParabolicBedFlowline
from oggm.core.massbalance import LinearMassBalance
from oggm import cfg
cfg.initialize()

# There are several solvers in OGGM core. We use the default one for this experiment
FlowlineModel = partial(FluxBasedModel, min_dt=0, cfl_number=0.01)

and some other useful packages

In [ ]:
import numpy as np

## Define model domain

In [ ]:
# number of steps from bottem to top of glacier
nx = 200

# model grid spacing in m
map_dx = 100

# distance along glacier (x-axis of glacier profil plot) in km
distance_along_glacier = np.linspace(0, nx, nx) * map_dx * 1e-3

# glacier top height
glacier_top_height = 4000

# glacier bottom height
glacier_bottom_height = 0

## Define default values for parameters and range for menu sliders

In [ ]:
# ELA height
ELA_height_min = 0
ELA_height_max = 4000
ELA_height_default = 3000
ELA_height_step = 100

# mass balance gradient
mb_gradient_max = 15
mb_gradient_min = 1
mb_gradient_default = 4
mb_gradient_step = 1

# years of calculation
years_min = 10
years_max = 1000
years_default = 150
years_step = 10

# linear bed rock slope
bed_rock_slope_values = ['steepest', 'steeper',
                         'steep', 'flat', 'flatter', 'flattest']
bed_rock_slope_default = 'flat'

# bed rock profile
bed_rock_profile_values = [
    'linear', 'getting flatter', 'getting steeper', 'cliff']
bed_rock_profile_default = 'linear'

# bed rock width
bed_rock_width_values = ['constant', 'getting narrower', 'getting wider',
                         'wide at the top and narrow at the bottom']
bed_rock_width_default = 'constant'

# Glen's creep parameter
glens_creep_parameter_values = ['small', 'medium', 'large']
glens_creep_parameter_default = 'medium'

# sliding parameter
sliding_parameter_values = ['no sliding', 'sliding']
sliding_parameter_default = 'no sliding'

# model dyears (how often the model state should be shown)
dyears_model_start = 1
dyears_model_end = 100
dyears_model_step = 1
dyears_model_default = 10

# maximum calculation years of model (before it gets interrupted)
max_calc_years_start = 100
max_calc_years_end = 5000
max_calc_years_step = 100
max_calc_years_default = 1000

## Define global model variables:

In [ ]:
bed_h = np.linspace(glacier_top_height, 0, nx)
mb_model = []
model = []
run_surface_heights = []
glacier_outlines = []
glen_a = cfg.PARAMS['glen_a']
fs = 0

# define width, so glacier bed could be created when plot is created (after creation model is ready)
widths = np.zeros(nx) + 4.

# years after the model progress should be shown
dyears = dyears_model_default

# years after model should be aborted
max_calc_years = max_calc_years_default

## Define some plotting variables

In [ ]:
# default tools for plots
default_tools_geometry = ['save', 'box_zoom', 'reset']
default_tools_timeseries = ['pan', 'save', 'box_zoom',
                            'xwheel_zoom', 'hover', 'reset']
# is needed so xwheel_zoom is activated by default
default_active_tools_timeseries = ['xwheel_zoom']

# Variable to follow which text is currently displayed in the headerline
header_text_status = 0

# colormap for velocity and thickness
colormap = ['#5ebaff', '#00faf4', '#ffffcc',
            '#ffe775', '#ffc140', '#ff8f20', '#ff6060']

# define nan array for glacier outline plotting in width plot
nan_array = np.empty(nx)
nan_array.fill(np.nan)

# index to force a 'change' for geometry figure
geometry_figure_change_index = 1

# define variables for tracking change in glacier volume and length
glacier_volume = [0]
glacier_length = [0]

# define variable for tracking time corresponding to glacier volume and length
glacier_time = [0]

# index to force a 'change' for timeseries figure (dynamic volume and length plot)
timeseries_figure_change_index = 1

# define values for table
years_table = []
ELA_table = []
mb_gradient_table = []
glen_a_table = []
fs_table = []
volume_table = []
length_table = []

# index to force a 'change' for timeseries dynamic map
timeseries_table_change_index = 1

# variable to store vertical lines position in volume and length plot
timeseries_vlin_pos = [0]

# for the ELA position in the width plot
ELA_x_position = 0

## Define global curves (elements of subplots)

In [ ]:
bed_rock_height_curve = []
bed_rock_width_curve = []
glacier_height_curve = []
mb_curve = []
width_curve = []
info_text = []
glacier_volume_curve = []
glacier_length_curve = []
timeseries_table = []

## Define Headerline with help functionality

Define appearance of Headerline and also implement the help functionality, so the help text is displayed in the Headerline

### help functions for Headerline buttons

In [ ]:
def next_help_button_click(arg=None):
    global header_text_status

    header_text_status += 1

    help_function()


def previous_help_button_click(arg=None):
    global header_text_status

    header_text_status -= 1

    help_function()

### define elements of Headerline

In [ ]:
header_text_default = ('<div style="font-size:20px">OGGM edu glacier simulator</div>' +
                       'With this app you can simulate idealized glaciers using the ' +
                       '<a href="https://oggm.org">OGGM</a> model. If you are not familiar with glaciers at all ' +
                       'or using the app the first time ' +
                       'click on "next" to go through an introduction. <br>This app was realised ' +
                       'using tools of <a href="https://holoviz.org">HoloViz</a>.')
header_text = pn.pane.Markdown(
    header_text_default, sizing_mode='stretch_width')

oggm_edu_logo = '<a href="http://edu.oggm.org"><img src="http://edu.oggm.org/en/latest/_images/oggm.gif" width=170></a>'
hv_logo = '<a href="https://holoviz.org"><img src="https://holoviz.org/assets/holoviz-logo-stacked.svg" width=80></a>'

next_help_button = pn.widgets.Button(disabled=False,
                                     name='next',
                                     button_type='success',
                                     width=75)
next_help_button.param.watch(next_help_button_click, 'clicks')

previous_help_button = pn.widgets.Button(disabled=True,
                                         name='previous',
                                         button_type='danger',
                                         width=75)
previous_help_button.param.watch(previous_help_button_click, 'clicks')

### Put actual Headerline together

In [ ]:
header = pn.Row(pn.Pane(oggm_edu_logo),
                pn.layout.Spacer(width=10),
                header_text,
                pn.Column(pn.layout.Spacer(),
                          previous_help_button,
                          next_help_button),
                pn.layout.Spacer(),
                pn.Pane(hv_logo, width=80), height=120, sizing_mode='stretch_width')

### define actual help function with help text

In [ ]:
def help_function():
    global header_text_status

    if header_text_status == 0:
        header_text.object = header_text_default
        previous_help_button.disabled = True

    elif header_text_status == 1:
        header_text.object = ('<div style="font-size:20px">OGGM edu glacier simulator beginner tour</div>' +
                              'Welcome to the beginner tour of the <a href="http://edu.oggm.org">OGGM edu</a> ' +
                              'glacier simulater. To navigate through the tour use the "next" button to the right ' +
                              'to continue the tour or "previous" to see the last page again. So lets start with a ' +
                              'click on "next"!')
        previous_help_button.disabled = False

    elif header_text_status == 2:
        menu.active = 0
        header_text.object = ('First let\'s have a look at the figures on the left side. The top figure shows ' +
                              'the glacierbed from the side in gray. The dark blue line shows the current glacier ' +
                              'state and you can see at the beginning this line follows the glacierbed, so there ' +
                              'is no glacier. The figure below shows the glacier from above and the dark gray areas ' +
                              'indicate the boarder of the glacierbed. So now click on the "run the model" ' +
                              'button to see the glacier grow and then on "next".')

    elif header_text_status == 3:
        header_text.object = ('So now you can see the thickness of the glacier in the top figure and the length of ' +
                              'the glacier in the bottom figure. If you want to zoom in you can do this by clicking ' +
                              'into one of the figures, hold the click and draw an rectangle. You can see that if ' +
                              'you zoom in into one figure the others automatically rezoom to the same extent. ' +
                              'To go back to the original extent click on the "Reset" button to the right of the ' +
                              'figures (two arrows forming a circle). After playing around with the zoom click ' +
                              'on "next".')

    elif header_text_status == 4:
        header_text.object = ('Now we are concentrating on the figure in the top right corner. This figure shows ' +
                              'the height dependent annual mass balance. But what is the annual mass balance? ' +
                              'The unit is meter per year, so when the annual mass balance at a specific height ' +
                              'is positive the glacier had gain mass during one year, so in other words ' +
                              'there were more accumulation (mass gain through e.g. snowfall or wind drift) ' +
                              'than ablation (mass loss through e.g. melting or wind drift) happening. Such a positive ' +
                              'annual mass balance is indicated with the green area in the figure and red indicates ' +
                              'a mass loss. ("next")')

    elif header_text_status == 5:
        header_text.object = ('In between the green (mass gain) and the red area (mass loss) you can see a dashed ' +
                              'black line with the annotation ELA, which stands for Equilibrium Line Altitude. ' +
                              'At this altitude the glacier do not change his mass during one year. And the black ' +
                              'solid line shows the actual mass balance for each altitude band. Through the glacier ' +
                              'flow mass is transported from higher altitudes down. For a more detailed ' +
                              'explenation of the mass balance concept you can click on this ' +
                              '<a href=http://www.antarcticglaciers.org/glacier-processes/introduction-glacier-mass-' +
                              'balance/#SECTION_1>link</a>. ("next")')

    elif header_text_status == 6:
        header_text.object = ('Summarizing the annual mass balance determines the ' +
                              '"behavior" of the glacier, e.g. how large can it become, at which altitudes the glacier ' +
                              'can survive. So always you click on the "run the model" button the model is starting ' +
                              'from zero and develops the glacier until it is in equilibrium with annual mass balance, ' +
                              'this means the glacier geometry is not changing anymore, although internal the glacier ' +
                              'is still moving mass from higher altitudes to lower ones. So now we start playing ' +
                              'around with the model and look how different parameters change the geometry of the ' +
                              'glacier. ("next")')
        next_help_button.disabled = False

    elif header_text_status == 7:
        header_text.object = ('You can see that there are three parameters to play with in the beginners mode: the ' +
                              'slope of the bedrock, the width of the bedrock and the ELA (equilibrium line altitude). ' +
                              'At the beginning you should only change one parameter to see how it effects the glacier.' +
                              '(end of Tour until now, more will come soon)')
        next_help_button.disabled = True

## Define Panels for tab menu

Defining the individual menus and putting them together in an tab menu.

In [ ]:
# define background color of tab menu
menu_background = '#fafafa'

### Panel for beginner mode

beginner button function

In [ ]:
def beginner_button_click(arg=None):
    # set Variables not changeable in beginner mode to there default values
    mb_gradient.value = mb_gradient_default
    bed_rock_profile.value = bed_rock_profile_default
    glens_creep_parameter.value = glens_creep_parameter_default
    sliding_parameter.value = sliding_parameter_default

    # for consitency set values of the advanced mode the same as choosen in beginner mode
    ELA_height_advanced.value = ELA_height.value
    bed_rock_width_advanced.value = bed_rock_width.value
    bed_rock_slope_advanced.value = bed_rock_slope.value

    # set numeric values out of selection
    set_glen_a()
    set_fs()

    run_the_model(stage='beginner mode')

actual beginner Panel

In [ ]:
bed_rock_slope = pn.widgets.Select(name='slope of bedrock profile',
                                   options=bed_rock_slope_values,
                                   value=bed_rock_slope_default,
                                   sizing_mode='stretch_width')

bed_rock_width = pn.widgets.Select(name='width along glacier',
                                   options=bed_rock_width_values,
                                   value=bed_rock_width_default,
                                   sizing_mode='stretch_width')

ELA_height = pn.widgets.IntSlider(name='ELA (equilibrium line altitude)',
                                  start=ELA_height_min,
                                  end=ELA_height_max,
                                  step=ELA_height_step,
                                  value=ELA_height_default,
                                  sizing_mode='stretch_width')

beginner_button = pn.widgets.Button(name='run the model',
                                    button_type='primary',
                                    margin=(15, 0, 0, 0))
beginner_button.param.watch(beginner_button_click, 'clicks')

beginner_panel = pn.Row(bed_rock_slope,
                        bed_rock_width,
                        ELA_height,
                        beginner_button,
                        background=menu_background,
                        sizing_mode='stretch_width')

### Panel for advanced mode

advanced button help functions

In [ ]:
def advance_model_click():
    # for consitency set values of the beginner mode the same as choosen in advanced mode
    ELA_height.value = ELA_height_advanced.value
    bed_rock_width.value = bed_rock_width_advanced.value
    bed_rock_slope.value = bed_rock_slope_advanced.value

    # set numeric values out of selection
    set_glen_a()
    set_fs()

advanced button functions

In [ ]:
def run_model_equilibrium_button_click(arg=None):
    advance_model_click()
    run_the_model(stage='advance equilibrium')


def run_model_years_button_click(arg=None):
    advance_model_click()
    run_the_model(stage='advance years')


def create_new_model_button_click(arg=None):
    global geometry_figure_change_index

    # set sliders of beginner mode to the same as choosen in advanced mode
    bed_rock_width.value = bed_rock_width_advanced.value
    bed_rock_slope.value = bed_rock_slope_advanced.value

    init_model()

actual advanced Panel

In [ ]:
mb_gradient = pn.widgets.IntSlider(name='mass balance gradient',
                                   start=mb_gradient_min,
                                   end=mb_gradient_max,
                                   step=mb_gradient_step,
                                   value=mb_gradient_default,
                                   sizing_mode='stretch_width')

bed_rock_profile = pn.widgets.Select(name='bedrock profile',
                                     options=bed_rock_profile_values,
                                     value=bed_rock_profile_default,
                                     sizing_mode='stretch_width')

ELA_height_advanced = pn.widgets.IntSlider(name='ELA',
                                           start=ELA_height_min,
                                           end=ELA_height_max,
                                           step=ELA_height_step,
                                           value=ELA_height_default,
                                           sizing_mode='stretch_width')

bed_rock_slope_advanced = pn.widgets.Select(name='slope of linear bedrock profile',
                                            options=bed_rock_slope_values,
                                            value=bed_rock_slope_default,
                                            sizing_mode='stretch_width')

bed_rock_width_advanced = pn.widgets.Select(name='width along glacier',
                                            options=bed_rock_width_values,
                                            value=bed_rock_width_default,
                                            sizing_mode='stretch_width')

glens_creep_parameter = pn.widgets.Select(name="Glen's creep parameter",
                                          options=glens_creep_parameter_values,
                                          value=glens_creep_parameter_default,
                                          sizing_mode='stretch_width')

sliding_parameter = pn.widgets.RadioBoxGroup(name='Sliding parameter',
                                             options=sliding_parameter_values,
                                             value=sliding_parameter_default,
                                             margin=(10, 0, 0, 0),
                                             sizing_mode='stretch_width')

run_model_years = pn.widgets.IntSlider(name='years to advance model',
                                       start=years_min,
                                       end=years_max,
                                       step=years_step,
                                       value=years_default,
                                       margin=(5, 0, 0, 0),
                                       sizing_mode='stretch_width')

run_model_equilibrium_button = pn.widgets.Button(name='run to equilibrium',
                                                 button_type='primary',
                                                 sizing_mode='stretch_width')
run_model_equilibrium_button.param.watch(
    run_model_equilibrium_button_click, 'clicks')

run_model_years_button = pn.widgets.Button(name='advance model',
                                           button_type='primary',
                                           sizing_mode='stretch_width')
run_model_years_button.param.watch(run_model_years_button_click, 'clicks')

create_new_model_button = pn.widgets.Button(name='create new model',
                                            button_type='primary',
                                            sizing_mode='stretch_width',
                                            margin=(15, 0, 0, 0))
create_new_model_button.param.watch(create_new_model_button_click, 'clicks')

advanced_panel = pn.Tabs(('run the model',
                          pn.Row(ELA_height_advanced,
                                 mb_gradient,
                                 glens_creep_parameter,
                                 pn.Row(sliding_parameter,
                                        width=90),
                                 run_model_years,
                                 pn.Column(run_model_years_button,
                                           run_model_equilibrium_button,
                                           sizing_mode='stretch_width'),
                                 sizing_mode='stretch_width')),
                         ('create new model',
                          pn.Row(bed_rock_profile,
                                 bed_rock_slope_advanced,
                                 bed_rock_width_advanced,
                                 create_new_model_button,
                                 sizing_mode='stretch_width')),
                         background=menu_background,
                         tabs_location='left')

### Panel for help

help functions for buttons

In [ ]:
def beginner_introduction_button_click(arg=None):
    global header_text_status

    # just to be sure the buttons are activated
    next_help_button.disabled = False
    previous_help_button.disabled = False

    # choose the start page of the beginner tour
    header_text_status = 1
    help_function()


def advanced_introduction_button_click(arg=None):
    pass


def show_subfigure_explenation_button_click(arg=None):
    pass


def show_parameter_explenation_button_click(arg=None):
    pass

actual help Panel

In [ ]:
help_intro_text = pn.pane.Markdown('<p style="margin-top: 0px;">Here you can jump to different sections ' +
                                   'of the help tour. The help will be shown in the Headerline and you ' +
                                   'can navigate through with the "next" and "previous" button.</p>',
                                   sizing_mode='stretch_width', height=20)

beginner_introduction_button = pn.widgets.Button(disabled=False,
                                                 name='beginner mode tour',
                                                 button_type='primary',
                                                 sizing_mode='stretch_width')
beginner_introduction_button.param.watch(
    beginner_introduction_button_click, 'clicks')

advanced_introduction_button = pn.widgets.Button(disabled=True,
                                                 name='advanced mode introduction',
                                                 button_type='primary',
                                                 sizing_mode='stretch_width')
advanced_introduction_button.param.watch(
    advanced_introduction_button_click, 'clicks')

show_subfigure_explenation_button = pn.widgets.Button(disabled=True,
                                                      name='show explenation of each subfigure',
                                                      button_type='primary',
                                                      sizing_mode='stretch_width')
show_subfigure_explenation_button.param.watch(
    show_subfigure_explenation_button_click, 'clicks')

show_parameter_explenation_button = pn.widgets.Button(disabled=True,
                                                      name='show explenation of parameters',
                                                      button_type='primary',
                                                      sizing_mode='stretch_width')
show_parameter_explenation_button.param.watch(
    show_parameter_explenation_button_click, 'clicks')

help_panel = pn.Row(help_intro_text,
                    pn.Column(beginner_introduction_button,
                              advanced_introduction_button),
                    pn.Column(show_subfigure_explenation_button,
                              show_parameter_explenation_button),
                    background=menu_background,
                    sizing_mode='stretch_width')

### Panel for color options

help functions

In [ ]:
def show_velocity_changed(arg=None):
    global geometry_figure_change_index
    set_glacier_height_curve()
    geometry_figure.event(count=geometry_figure_change_index)
    geometry_figure_change_index += 1


def show_thickness_changed(arg=None):
    global geometry_figure_change_index
    set_width_curve()
    geometry_figure.event(count=geometry_figure_change_index)
    geometry_figure_change_index += 1

actual panel

In [ ]:
show_velocity = pn.widgets.Checkbox(name='show velocity as colors (top left figure)',
                                    sizing_mode='stretch_width',
                                    margin=(2, 0, 0, 0))
show_velocity.param.watch(show_velocity_changed, 'value')

show_thickness = pn.widgets.Checkbox(name='show thickness as colors (bottom left figure)',
                                     sizing_mode='stretch_width',
                                     margin=(0, 0))
show_thickness.param.watch(show_thickness_changed, 'value')

color_options = pn.pane.Markdown(('<p style="margin-top: 0px;">Here you can choose if the velocity or/and the thickness ' +
                                  'of the ice should be shown. The colorbar shows the intervals, ' +
                                  'maximum refers to the value shown in the "info text".</p> '),
                                 height=41,
                                 sizing_mode='stretch_width',
                                 margin=0)


# is needed to directly get to bokeh styling options
def colorbar_hook(plot, element):
    plot.handles['plot'].border_fill_color = menu_background
    plot.handles['plot'].min_border_right = 50
    plot.handles['plot'].min_border_bottom = 0
    plot.handles['colorbar'].background_fill_color = menu_background
    plot.handles['colorbar'].height = 7


ColorBar = hv.Polygons({'x': [],
                        'y': [],
                        'value': [0]},
                       vdims='value').opts(hooks=[colorbar_hook],
                                           responsive=True,
                                           colorbar_position='top',
                                           colorbar_opts={'orientation': 'horizontal',
                                                          'major_label_overrides': {-1: '0',
                                                                                    -0.5: 'quater',
                                                                                    0: 'half',
                                                                                    0.5: 'three-quaters',
                                                                                    1: 'maximum'},
                                                          'major_label_text_font_size': '10pt'},
                                           yaxis=None,
                                           xaxis=None,
                                           # show_frame=False,
                                           toolbar=None,
                                           frame_height=0,
                                           cmap=colormap,
                                           colorbar=True)

color_option_panel = pn.Column(pn.Row(color_options,
                                      pn.Column(show_velocity,
                                                show_thickness,
                                                background=menu_background,
                                                sizing_mode='stretch_width')),
                               pn.Row(ColorBar,
                                      height=40,
                                      sizing_mode='stretch_width'),
                               background=menu_background,
                               sizing_mode='stretch_width')

### Panel for model options

help function for button click

In [ ]:
def set_model_options(arg=None):
    global dyears
    global max_calc_years

    dyears = dyears_model_slider.value
    max_calc_years = max_calc_years_slider.value
    current_model_options.object = current_model_options_text.format(
        dyears_model_slider.value, max_calc_years_slider.value)

actual panel

In [ ]:
dyears_model_slider = pn.widgets.IntSlider(name='show model progress every ... years',
                                           start=dyears_model_start,
                                           end=dyears_model_end,
                                           step=dyears_model_step,
                                           value=dyears_model_default,
                                           sizing_mode='stretch_width')

max_calc_years_slider = pn.widgets.IntSlider(name='maximum calculation year of model',
                                             start=max_calc_years_start,
                                             end=max_calc_years_end,
                                             step=max_calc_years_step,
                                             value=max_calc_years_default,
                                             sizing_mode='stretch_width')

current_model_options_text = ('Model currently showing progress every **{}** years and ' +
                              'abort model after **{}** years of calculation')
current_model_options = pn.pane.Markdown(current_model_options_text.format(dyears_model_default, max_calc_years_default),
                                         sizing_mode='stretch_width')

set_model_options_button = pn.widgets.Button(name='set new model options',
                                             button_type='primary',
                                             sizing_mode='stretch_width')
set_model_options_button.param.watch(set_model_options, 'clicks')

model_option_panel = pn.Row(current_model_options,
                            dyears_model_slider,
                            max_calc_years_slider,
                            set_model_options_button,
                            background=menu_background,
                            sizing_mode='stretch_width')

### Panel to choose used figures

In [ ]:
# functions to set an infotext in the plots telling if they 
# are active or not
def switch_geometry_plot(arg=None):
    global geometry_figure_change_index
    if show_geometry_plot.value:
        set_info_text(stage='switched on')
    else:
        set_info_text(stage='switched off')
    geometry_figure.event(count=geometry_figure_change_index)
    geometry_figure_change_index += 1


def switch_timeseries_plot(arg=None):
    global timeseries_figure_change_index
    global timeseries_table_change_index

    init_timeseries_plot()

    timeseries_figure.event(count=timeseries_figure_change_index)
    timeseries_figure_change_index += 1
    timeseries_table.event(count=timeseries_table_change_index)
    timeseries_table_change_index += 1

In [ ]:
show_geometry_plot = pn.widgets.Checkbox(
    name='show geometry plot', value=True)
show_geometry_plot.param.watch(switch_geometry_plot, 'value')

show_timeseries_plot = pn.widgets.Checkbox(
    name='show timeseries plot', value=False)
show_timeseries_plot.param.watch(switch_timeseries_plot, 'value')

choose_plots_panel = pn.Column(show_geometry_plot,
                               show_timeseries_plot,
                               background=menu_background,
                               sizing_mode='stretch_width')

### add all panels together as tabs

In [ ]:
menu = pn.Tabs(('beginner mode', beginner_panel),
               ('advanced mode', advanced_panel),
               ('color options', color_option_panel),
               ('model options', model_option_panel),
               ('choose plots', choose_plots_panel),
               ('find help here', help_panel),
               background=menu_background)

## Define functions to set model parameters

Functions to set model parameters, which are needed to initialize a new model

### set bed h

In [ ]:
def set_bed_h():
    global bed_h

    # look which bed rock profile was chosen
    if bed_rock_profile.value == 'linear':
        # look which slope was choosen and set the heigth at each point along the glacier
        if bed_rock_slope.value == 'steepest':
            bed_h = np.append(np.linspace(glacier_top_height, 0, np.round(nx / 4)),
                              np.zeros(nx - np.int(np.round(nx / 4))))
        elif bed_rock_slope.value == 'steeper':
            bed_h = np.append(np.linspace(glacier_top_height, 0, np.round(nx / 4) * 2),
                              np.zeros(nx - np.int(np.round(nx / 4) * 2)))
        elif bed_rock_slope.value == 'steep':
            bed_h = np.append(np.linspace(glacier_top_height, 0, np.round(nx / 4) * 3),
                              np.zeros(nx - np.int(np.round(nx / 4) * 3)))
        elif bed_rock_slope.value == 'flat':
            bed_h = np.linspace(glacier_top_height, 0, nx)
        elif bed_rock_slope.value == 'flatter':
            bed_h = np.linspace(glacier_top_height, 1500, nx)
        elif bed_rock_slope.value == 'flattest':
            bed_h = np.linspace(glacier_top_height, 3000, nx)

    elif bed_rock_profile.value == 'getting flatter':
        bed_h = np.array([(np.log(x / nx + 0.03) / np.log(0.5) * 0.2 + 0.03) *
                          glacier_top_height for x in np.arange(nx)])

    elif bed_rock_profile.value == 'getting steeper':
        bed_h = np.array([(np.log(-x / nx / 4 + 0.26) * 0.2 / np.log(2) + 1.3) *
                          glacier_top_height for x in np.arange(nx)])

    elif bed_rock_profile.value == 'cliff':
        # define extend of cliff
        cliff_top = 2500
        cliff_bottom = 2400

        bed_h = np.concatenate((np.linspace(glacier_top_height, cliff_top, nx/2),
                                np.linspace(cliff_bottom, glacier_bottom_height, nx/2)))

### set widths

In [ ]:
def set_widths():
    global widths

    # look which glacier width was selected
    if bed_rock_width.value == 'constant':
        widths = np.zeros(nx) + 4.

    elif bed_rock_width.value == 'getting wider':
        widths = np.array([(2 / nx * x + 0.5) * 2 for x in np.arange(nx)])

    elif bed_rock_width.value == 'getting narrower':
        widths = np.array([(-2 / nx * x + 2.5) * 2 for x in np.arange(nx)])

    elif bed_rock_width.value == 'wide at the top and narrow at the bottom':
        widths = np.append(np.ones(np.int(np.round(nx/5))) * 4,
                           np.ones(np.int(nx - np.round(nx/5))) * 2)

### set glen a

In [ ]:
def set_glen_a():
    global glen_a

    if glens_creep_parameter.value == 'medium':
        glen_a = cfg.PARAMS['glen_a']
    elif glens_creep_parameter.value == 'small':
        glen_a = cfg.PARAMS['glen_a'] / 10
    elif glens_creep_parameter.value == 'large':
        glen_a = cfg.PARAMS['glen_a'] * 10

### set fs

In [ ]:
def set_fs():
    global fs

    if sliding_parameter.value == 'no sliding':
        fs = 0
    elif sliding_parameter.value == 'sliding':
        fs = 5.7e-20

### set mb model

In [ ]:
def set_mb_model():
    global mb_model

    ELA = int(ELA_height.value)
    gradient = int(mb_gradient.value)

    mb_model = LinearMassBalance(ELA, grad=gradient)

## Define functions to initialize a model and plots

Functions initialize a new model (set all parameters and define new `FlowlineModel`) and also set all plots to there default appearance

### init model

In [ ]:
def init_model():
    global model

    # set all parameters for the model according to the choosen parameters from the menu
    set_mb_model()
    set_bed_h()
    set_widths()
    set_glen_a()
    set_fs()

    # set glacier surface height to bed height because at the beginning there is no glacier
    surface_h = bed_h

    # initialize flowline with rectangular shape of glacier
    model_flowline = RectangularBedFlowline(surface_h=surface_h, bed_h=bed_h,
                                            widths=widths, map_dx=map_dx)

    # initialize the actual model
    model = FlowlineModel(model_flowline, mb_model=mb_model, y0=0., glen_a=glen_a,
                          fs=fs, check_for_boundaries=False)

    # initialize all plots with new model
    init_geometry_plot()
    init_timeseries_plot()

### init geometry plot

In [ ]:
def init_geometry_plot():
    global run_surface_heights
    global glacier_outlines
    global geometry_figure_change_index

    # set surface height for plotting, at beginn old and new one are the same
    run_surface_heights = np.repeat(
        np.array(model.fls[-1].surface_h, ndmin=2), 2, axis=0)

    # set glacier outlines for plotting, at beginn old and new one are the same
    glacier_outlines = np.repeat(np.array(
        np.where(model.fls[-1].thick > 0, widths, nan_array), ndmin=2), 2, axis=0)

    # set all subplots for the geometry plot
    if show_geometry_plot.value:
        set_info_text(stage='new model')
    else:
        set_info_text(stage='switched off')
    set_bed_rock_curves()
    set_glacier_height_curve()
    set_mb_curve()
    set_width_curve()

    # force to refresh the geometry plot, the try statement is needed for the first initialization,
    # when the dynamic map do not yet exist
    try:
        geometry_figure.event(count=geometry_figure_change_index)
        geometry_figure_change_index += 1
    except NameError:
        pass

### init timeseries plot

In [ ]:
def init_timeseries_plot():
    global glacier_volume
    global glacier_length
    global glacier_time
    global timeseries_vlin_pos
    global years_table
    global ELA_table
    global mb_gradient_table
    global glen_a_table
    global fs_table
    global volume_table
    global length_table
    global timeseries_table_change_index
    global timeseries_figure_change_index

    # set parameters for volume and length plot back to default
    glacier_volume = [0]
    glacier_length = [0]
    glacier_time = [0]
    timeseries_vlin_pos = [0]

    # change the volume and length subplots
    set_glacier_volume_curve()
    set_glacier_length_curve()

    # empty table entries back
    years_table = []
    ELA_table = []
    mb_gradient_table = []
    glen_a_table = []
    fs_table = []
    volume_table = []
    length_table = []

    # set the new values in the table
    set_timeseries_table()

    # force the timeseries table to refresh, the try statement is needed for the first initialization,
    # when the dynamic map do not yet exist
    try:
        timeseries_table_DynamicMap.event(count=timeseries_table_change_index)
        timeseries_table_change_index += 1
    except NameError:
        pass

    # force the timeseries figure to refresh, the try statement is needed for the first initialization,
    # when the dynamic map do not yet exist
    try:
        timeseries_figure.event(count=timeseries_figure_change_index)
        timeseries_figure_change_index += 1
    except NameError:
        pass

## Define function for creating curves

Functions to determine the look of each subplot or subplot element. After the definition the Functions are also called so all subplots are set to the default looking.

### set bed rock curves

In [ ]:
def set_bed_rock_curves():
    global bed_rock_height_curve
    global bed_rock_width_curve

    # create curve of bed rock for view from beside (top left subfigure, geometry plot)
    bed_rock_height_curve = hv.Area((distance_along_glacier, bed_h),
                                    'distance along glacier (km)', 'altitude (m)',
                                    ).opts(default_tools=default_tools_geometry,
                                           color='lightgray',
                                           line_alpha=0)

    # create curve of bed rock for view from above (bottom left subfigure, geometry plot)
    bed_rock_width_curve = hv.Area((distance_along_glacier, -widths/2, widths/2),
                                   kdims=['distance along glacier (km)'],
                                   vdims=['y', 'y2']
                                   # label='glacier rock bed'
                                   ).redim.label(x='distance along glacier (km)',
                                                 y='width (m)').opts(default_tools=default_tools_geometry,
                                                                     color='lightgray',
                                                                     line_alpha=0)

### set glacier height curve

In [ ]:
def set_glacier_height_curve():
    global glacier_height_curve

    ELA = int(ELA_height.value)

    if not np.array_equal(run_surface_heights[0], bed_h):
        glacier_height_curve = (hv.Area((distance_along_glacier, run_surface_heights[0]),
                                        'distance along glacier (km)', 'altitude (m)',
                                        # label='last'
                                        ).opts(default_tools=default_tools_geometry,
                                               line_dash='dashed',
                                               line_color='darkgray',
                                               color='white',
                                               alpha=0.6,
                                               line_alpha=1,
                                               line_width=3) *
                                hv.Area((distance_along_glacier, run_surface_heights[1]),
                                        'distance along glacier (km)', 'altitude (m)',
                                        label='glacier'
                                        ).opts(default_tools=default_tools_geometry,
                                               color='white',
                                               line_color='blue',
                                               alpha=0.6,
                                               line_alpha=1,
                                               line_width=3))
    else:
        glacier_height_curve = (hv.Area(([], []),
                                        'distance along glacier (km)', 'altitude (m)',
                                        # label='last'
                                        ).opts(default_tools=default_tools_geometry,
                                               line_dash='dashed',
                                               line_color='darkgray',
                                               color='white',
                                               alpha=0.6,
                                               line_alpha=1,
                                               line_width=3) *
                                hv.Area((distance_along_glacier, run_surface_heights[1]),
                                        'distance along glacier (km)', 'altitude (m)',
                                        label='glacier'
                                        ).opts(default_tools=default_tools_geometry,
                                               color='white',
                                               line_color='blue',
                                               alpha=1,
                                               line_alpha=1,
                                               line_width=3))

     # looking for potential segments which can be colored if wanted
    index_of_segments = np.arange(np.sum(model.fls[-1].thick > 0))

    # show ice velocity in colors if wanted and if there is something to show
    if show_velocity.value and index_of_segments.size != 0:
        # calculate the mean velocity between two segments
        plot_velocity = [(np.array(model.get_diagnostics()['ice_velocity'])[i] +
                          np.array(model.get_diagnostics()['ice_velocity'])[i + 1]) / 2
                         for i in index_of_segments]

        # try statement is necessery because there are some problems with 'linear_cmap'
        # when the app is started in the notebook, and it works when the app is shown
        # in the browser seperatly
        try:
            # is necessary so the line_color is the same as the filled color of the polygons
            velocity_mapper = []
            velocity_mapper = linear_cmap(field_name='velocity', palette=colormap,
                                          low=np.min(plot_velocity),
                                          high=np.max(plot_velocity))

            # actual colored velocity
            glacier_height_curve *= hv.Polygons([{'x': [distance_along_glacier[i],
                                                        distance_along_glacier[i + 1],
                                                        distance_along_glacier[i + 1],
                                                        distance_along_glacier[i]],
                                                  'y': [bed_h[i],  # 0,
                                                        bed_h[i+1],  # 0,
                                                        run_surface_heights[1][i + 1],
                                                        run_surface_heights[1][i]],
                                                  'velocity': plot_velocity[i]}
                                                 for i in index_of_segments],
                                                vdims='velocity').opts(default_tools=default_tools_geometry,
                                                                       line_color=velocity_mapper,
                                                                       cmap=colormap)

        except:
            glacier_height_curve *= hv.Polygons([{'x': [distance_along_glacier[i],
                                                        distance_along_glacier[i + 1],
                                                        distance_along_glacier[i + 1],
                                                        distance_along_glacier[i]],
                                                  'y': [bed_h[i],  # 0,
                                                        bed_h[i+1],  # 0,
                                                        run_surface_heights[1][i + 1],
                                                        run_surface_heights[1][i]],
                                                  'velocity': plot_velocity[i]}
                                                 for i in index_of_segments],
                                                vdims='velocity').opts(default_tools=default_tools_geometry,
                                                                       line_color=None,
                                                                       cmap=colormap)

    else:
        # when there is no glacier or colors should not be shown
        # this is needed so bokeh knows about the Polygons element
        glacier_height_curve *= hv.Polygons([{'x': [],
                                              'y': [],
                                              'velocity': [0]}],
                                            vdims='velocity').opts(default_tools=default_tools_geometry)

    # add ELA to plot
    glacier_height_curve *= hv.Curve((distance_along_glacier, np.repeat(ELA, np.size(distance_along_glacier))),
                                     'distance along glacier (km)',
                                     'altitude (m)'
                                     ).opts(default_tools=default_tools_geometry,
                                            line_dash='dashed',
                                            line_width=3,
                                            color='black')

### set mb curve

In [ ]:
def set_mb_curve():
    global mb_curve

    ELA = int(ELA_height.value)
    
    # define colors for mass gain/loss
    mass_gain_color = 'green'
    mass_loss_color = 'red'

    # define the range of the mass balance curve wich should be shown
    x_range_of_mb_curve = np.array([-20, 20])
    y_range_of_mb_curve = np.array([0, 4500])

    # get the annual mass balance for the given range of the x axis
    annual_mb = mb_model.get_annual_mb(y_range_of_mb_curve) * cfg.SEC_IN_YEAR

    # get the index for the current glacier extent
    glacier_extent_index = np.sum(run_surface_heights[1]-bed_h > 0)

    if glacier_extent_index == 0:
        mb_curve = (hv.Area(([], [], []),
                            kdims=['annual mass balance (m/year)'],
                            # vdims have to be different
                            vdims=['altitude (m)', 'altitude (m) '],
                            label='mass gain'
                            ).opts(default_tools=default_tools_geometry,
                                   color=mass_gain_color,
                                   line_alpha=0) *
                    hv.Area(([], [], []),
                            kdims=['annual mass balance (m/year)'],
                            # vdims have to be different
                            vdims=['altitude (m)', 'altitude (m) '],
                            label='mass loss'
                            ).opts(default_tools=default_tools_geometry,
                                   color=mass_loss_color,
                                   line_alpha=0))
    elif run_surface_heights[1][glacier_extent_index] >= ELA:
        mb_curve = (hv.Area(([x_range_of_mb_curve],
                             [np.repeat(run_surface_heights[1]
                                        [glacier_extent_index], 2)],
                             [np.repeat(run_surface_heights[1][0], 2)]),
                            kdims=['annual mass balance (m/year)'],
                            # vdims have to be different
                            vdims=['altitude (m)', 'altitude (m) '],
                            label='mass gain'
                            ).opts(default_tools=default_tools_geometry,
                                   color=mass_gain_color,
                                   line_alpha=0) *
                    hv.Area(([], [], []),
                            kdims=['annual mass balance (m/year)'],
                            # vdims have to be different
                            vdims=['altitude (m)', 'altitude (m) '],
                            label=mass_loss_color
                            ).opts(default_tools=default_tools_geometry,
                                   color='lightcoral',
                                   line_alpha=0))
    else:
        mb_curve = (hv.Area(([x_range_of_mb_curve],
                             [np.repeat(ELA, 2)],
                             [np.repeat(run_surface_heights[1][0], 2)]),
                            kdims=['annual mass balance (m/year)'],
                            # vdims have to be different
                            vdims=['altitude (m)', 'altitude (m) '],
                            label='mass gain'
                            ).opts(default_tools=default_tools_geometry,
                                   color=mass_gain_color,
                                   line_alpha=0) *
                    hv.Area(([x_range_of_mb_curve],
                             [np.repeat(ELA, 2)],
                             [np.repeat(run_surface_heights[1]
                                        [glacier_extent_index], 2)]),
                            kdims=['annual mass balance (m/year)'],
                            # vdims have to be different
                            vdims=['altitude (m)', 'altitude (m) '],
                            label='mass loss'
                            ).opts(default_tools=default_tools_geometry,
                                   color=mass_loss_color,
                                   line_alpha=0))

    mb_curve *= (hv.Curve((annual_mb, y_range_of_mb_curve),
                          'annual mass balance (m/year)',
                          'altitude (m)',
                          label='mass balance'
                          ).opts(default_tools=default_tools_geometry,
                                 color='black') *
                 hv.Curve((x_range_of_mb_curve, np.repeat(ELA, 2)),
                          'annual mass balance (m/year)',
                          'altitude (m)',
                          label='ELA'
                          ).opts(default_tools=default_tools_geometry,
                                 line_dash='dashed',
                                 line_width=3,
                                 color='black'))

### set width curve

In [ ]:
def set_ELA_x_position():
    global ELA_x_position

    # find index for ELA position on x-axis (distance along glacier)
    ELA_x_position = (np.abs(bed_h - ELA_height.value)).argmin()

In [ ]:
def set_width_curve():
    global width_curve

    # set the ELA position on the x axis
    set_ELA_x_position()

    # define lower and upper y limit which is shown
    y_min = -3  # in m
    y_max = 3

    if not np.array_equal(run_surface_heights[0], bed_h):
        width_curve = (hv.Area((distance_along_glacier, -glacier_outlines[0]/2, glacier_outlines[0]/2),
                               kdims=['distance along glacier (km)'],
                               vdims=['y', 'y2']
                               # label='old'
                               ).redim.label(x='distance along glacier (km)',
                                             y='width (m)').opts(default_tools=default_tools_geometry,
                                                                 line_dash='dashed',
                                                                 color='white',
                                                                 line_color='darkgray',
                                                                 alpha=0.6,
                                                                 line_alpha=1,
                                                                 line_width=3) *
                       hv.Area((distance_along_glacier, -glacier_outlines[1]/2, glacier_outlines[1]/2),
                               kdims=['distance along glacier (km)'],
                               vdims=['y', 'y2']
                               # label='new'
                               ).redim.label(x='distance along glacier (km)',
                                             y='width (m)').opts(default_tools=default_tools_geometry,
                                                                 color='white',
                                                                 line_color='blue',
                                                                 alpha=0.6,
                                                                 line_alpha=1,
                                                                 line_width=3))
    else:
        width_curve = (hv.Area(([], [], []),
                               kdims=['distance along glacier (km)'],
                               vdims=['y', 'y2']
                               # label='old'
                               ).redim.label(x='distance along glacier (km)',
                                             y='width (m)').opts(default_tools=default_tools_geometry,
                                                                 line_dash='dashed',
                                                                 color='white',
                                                                 line_color='darkgray',
                                                                 alpha=0.6,
                                                                 line_alpha=1,
                                                                 line_width=3) *
                       hv.Area((distance_along_glacier, -glacier_outlines[1]/2, glacier_outlines[1]/2),
                               kdims=['distance along glacier (km)'],
                               vdims=['y', 'y2']
                               # label='new'
                               ).redim.label(x='distance along glacier (km)',
                                             y='width (m)').opts(default_tools=default_tools_geometry,
                                                                 color='white',
                                                                 line_color='blue',
                                                                 alpha=1,
                                                                 line_alpha=1,
                                                                 line_width=3))

    # looking for potential segments which can be colored if wanted
    index_of_segments = np.arange(np.sum(model.fls[-1].thick > 0))

    # show ice velocity in colors if wanted and there is something to show
    if show_thickness.value and index_of_segments.size != 0:
        # calculate the mean thickness between two segments
        plot_thickness = [(np.array(model.get_diagnostics()['ice_thick'])[i] +
                           np.array(model.get_diagnostics()['ice_thick'])[i + 1]) / 2
                          for i in index_of_segments]

        # try statement is necessery because there are some problems with 'linear_cmap'
        # when the app is started in the notebook, and it works when the app is shown
        # in the browser seperatly
        try:
            # is necessary so the line_color is the same as the filled color of the polygons
            thickness_mapper = linear_cmap(field_name='thickness', palette=colormap,
                                           low=np.min(plot_thickness),
                                           high=np.max(plot_thickness))

            # actual colored thickness
            width_curve *= hv.Polygons([{'x': [distance_along_glacier[i],
                                               distance_along_glacier[i + 1],
                                               distance_along_glacier[i + 1],
                                               distance_along_glacier[i]],
                                         'y': [-glacier_outlines[1][i]/2,
                                               -glacier_outlines[1][i + 1]/2,
                                               glacier_outlines[1][i + 1]/2,
                                               glacier_outlines[1][i]/2],
                                         'thickness': plot_thickness[i]}
                                        for i in index_of_segments],
                                       vdims='thickness').opts(default_tools=default_tools_geometry,
                                                               line_color=thickness_mapper,
                                                               cmap=colormap)
        except:
            width_curve *= hv.Polygons([{'x': [distance_along_glacier[i],
                                               distance_along_glacier[i + 1],
                                               distance_along_glacier[i + 1],
                                               distance_along_glacier[i]],
                                         'y': [-glacier_outlines[1][i]/2,
                                               -glacier_outlines[1][i + 1]/2,
                                               glacier_outlines[1][i + 1]/2,
                                               glacier_outlines[1][i]/2],
                                         'thickness': plot_thickness[i]}
                                        for i in index_of_segments],
                                       vdims='thickness').opts(default_tools=default_tools_geometry,
                                                               line_color=None,
                                                               cmap=colormap)
    else:
        # when there is no glacier or colors should not be shown
        # this is needed so bokeh knows about the Polygons element
        width_curve *= hv.Polygons([{'x': [],
                                     'y': [],
                                     'thickness': [0]}],
                                   vdims='thickness').opts(default_tools=default_tools_geometry)

    width_curve *= (hv.Curve((distance_along_glacier, np.repeat(0, np.size(distance_along_glacier))),
                             'distance along glacier (km)',
                             'width (m)'  # ,
                             # label='center Flowline'
                             ).opts(default_tools=default_tools_geometry,
                                    color='black',
                                    line_width=1) *
                    hv.Curve((np.repeat(distance_along_glacier[ELA_x_position], 2), [y_min, y_max]),
                             'distance along glacier (km)',
                             'width (m)',
                             # label='ELA'
                             ).opts(default_tools=default_tools_geometry,
                                    line_dash='dashed',
                                    line_width=3,
                                    color='black'))

    width_curve.opts(ylim=(y_min, y_max))

### set info text

In [ ]:
def set_info_text(stage='new model'):
    global info_text

    text_color = 'black'

    if stage == 'new model':
        text = 'New Model geometry\ninitialised'
    elif stage == 'switched off':
        text = 'This plot is currently not used/updated.\nTo switch it on go to "choose plots".'
        text_color = 'red'
    elif stage == 'switched on':
        text = 'Plot is switched on again\nand will be updated with the next run!'
        text_color = 'darkgreen'
    else:
        if stage == 'running model':
            text = 'Model is running: '
        elif stage == 'equilibrium':
            text = 'Model reached equilibrium '
        elif stage == 'stopped model':
            text = 'Model finished '
        elif stage == 'aborted model':
            text = 'Model aborted (max calculation time reached, see "model options")'

        text += '\nTime: {:4.0f} years          Volume: '

        if model.volume_km3 > 1:
            text += '{:9.2f} km³\n'
            volume = model.volume_km3
        else:
            text += '{:9.0f} m³\n'
            volume = model.volume_m3

        text += 'max ice thickness: {:.0f} m          max ice velocity: {:.0f} m/year'
        text = text.format(model.yr,
                           volume,
                           np.max(model.get_diagnostics()['ice_thick']),
                           np.max(model.get_diagnostics()['ice_velocity']) * cfg.SEC_IN_YEAR)

    info_text = hv.Text(0.5, 0.5, text).opts(default_tools=['save', 'reset'],
                                             show_frame=False,
                                             text_baseline='middle',
                                             text_align='center',
                                             text_font_size='10pt',
                                             text_font='courier',
                                             text_color=text_color,
                                             xaxis=None,
                                             yaxis=None)

### set glacier volume curve

In [ ]:
def set_glacier_volume_curve():
    global glacier_volume_curve

    glacier_volume_curve = hv.Curve((glacier_time, glacier_volume),
                                    'Time (years)',
                                    'volume (km³)'
                                    ).opts(default_tools=default_tools_timeseries,
                                           active_tools=default_active_tools_timeseries)

    # draws 11 helplines to seperate the sections
    for i in np.arange(11):
        # looks if the i'th line is needed, else the line will be drawn at zero
        try:
            vlin_pos = timeseries_vlin_pos[i]
        except IndexError:
            vlin_pos = 0

        # actual add the vertical lines
        glacier_volume_curve *= hv.VLine(vlin_pos).opts(default_tools=default_tools_timeseries,
                                                        active_tools=default_active_tools_timeseries,
                                                        color='black')

    # is needed for dynamical axes
    glacier_volume_curve.opts(opts.Curve(framewise=True))

### set glacier length curve

In [ ]:
def set_glacier_length_curve():
    global glacier_length_curve

    glacier_length_curve = hv.Curve((glacier_time, glacier_length),
                                    'Time (years)',
                                    'length (m)'
                                    ).opts(default_tools=default_tools_timeseries,
                                           active_tools=default_active_tools_timeseries)

    # variable to count how many 'empty' sections
    empty_sections_count = 0

    # draws 11 helplines to seperate the sections and add the section number
    for i in np.arange(11):
        # set the section number to default (if not needed)
        section_number_text = ' '
        section_number_x = 0
        section_number_y = 0

        # looks if the i'th line and text is needed
        try:
            vlin_pos = timeseries_vlin_pos[i]

            # looks if vertical line position is not zero and
            # then set the section number text and position
            if vlin_pos != 0:
                section_number_text = ('# ' + str(i - 1))
                section_number_x = ((timeseries_vlin_pos[i] -
                                     timeseries_vlin_pos[i - 1]) / 2 +
                                    timeseries_vlin_pos[i - 1])
                section_number_y = np.max(glacier_length) / 4
        except IndexError:
            vlin_pos = 0

        if vlin_pos == 0:
            empty_sections_count += 1

        # actual add the vertical lines
        glacier_length_curve *= hv.VLine(vlin_pos).opts(default_tools=default_tools_timeseries,
                                                        active_tools=default_active_tools_timeseries,
                                                        color='black')

        # actual add the section number
        glacier_length_curve *= hv.Text(section_number_x,
                                        section_number_y,
                                        section_number_text
                                        ).opts(default_tools=default_tools_timeseries,
                                               active_tools=default_active_tools_timeseries)

    # looks if the plot is activated and if there is already someting to show,
    # if there is nothing to show the text tells that the plot is ready
    if show_timeseries_plot.value and len(glacier_length) == 1:
        glacier_length_curve *= hv.Text(0,
                                        0,
                                        'plot is ready to use \nand update with the next run'
                                        ).opts(default_tools=default_tools_timeseries,
                                               active_tools=default_active_tools_timeseries,
                                               text_color='darkgreen')
    elif not show_timeseries_plot.value:
        glacier_length_curve *= hv.Text(0,
                                        0,
                                        'plot is switched off \nto switch on look at "choose plots"'
                                        ).opts(default_tools=default_tools_timeseries,
                                               active_tools=default_active_tools_timeseries,
                                               text_color='red'
                                               )
    else:
        glacier_length_curve *= hv.Text(0,
                                        0,
                                        ' '
                                        ).opts(default_tools=default_tools_timeseries,
                                               active_tools=default_active_tools_timeseries)

    # is needed for dynamical axes
    glacier_length_curve.opts(opts.Curve(framewise=True))

### set timeseries table

In [ ]:
def timeseries_table_hook(plot, element):
    plot.handles['table'].columns[0].width = 30
    plot.handles['table'].columns[1].width = 60
    plot.handles['table'].columns[2].width = 150
    plot.handles['table'].columns[3].width = 110
    plot.handles['table'].columns[4].width = 50
    plot.handles['table'].columns[5].width = 110
    plot.handles['table'].columns[6].width = 70


def set_timeseries_table():
    global timeseries_table

    timeseries_table = hv.Table((years_table, ELA_table, mb_gradient_table,
                                 glen_a_table, fs_table, volume_table,
                                 length_table),
                                ['Years', 'ELA (m)', 'gradient (m/year)',
                                 'Glen\'s creep', 'sliding'],
                                ['volume (km³)', 'length (m)']
                                ).opts(width=500,
                                       hooks=[timeseries_table_hook])

## Define function to run model

In [ ]:
def run_the_model(stage='beginner mode'):
    global model
    global run_surface_heights
    global glacier_outlines
    global timeseries_table_change_index
    global timeseries_figure_change_index
    global geometry_figure_change_index

    if stage == 'beginner mode':
        # initialize a new model
        init_model()

    elif stage == 'advance equilibrium' or stage == 'advance years':
        # initialize model with old extend and new parameters
        set_mb_model()
        set_glen_a()
        set_fs()
        model = FlowlineModel(model.fls[-1], mb_model=mb_model, y0=0.,
                              glen_a=glen_a, fs=fs, check_for_boundaries=False)

        # determine glacier state at starting point
        run_surface_heights = np.repeat(
            np.array(model.fls[-1].surface_h, ndmin=2), 2, axis=0)
        glacier_outlines = np.repeat(np.array(
            np.where(model.fls[-1].thick > 0, widths, nan_array), ndmin=2), 2, axis=0)

    if stage == 'beginner mode' or stage == 'advance equilibrium':
        # mostly same function as from OGGM FlowlineModel.run_until_equilibrium
        was_close_zero = 0
        t_rate = 1

        # first expression is the threshold for the volume change from one to the next calculated step,
        # second is the checking if the current model year is smaller than the choosen one,
        # third checks if the volume of the glacier is not always 0 (maximum five times)
        while (t_rate > 0.001) and (model.yr < max_calc_years) and (was_close_zero < 5):
            v_bef = model.volume_m3
            model.run_until(model.yr + dyears)
            v_af = model.volume_m3
            if np.isclose(v_bef, 0., atol=1):
                t_rate = 1
                was_close_zero += 1
            else:
                t_rate = np.abs(v_af - v_bef) / v_bef

            # check if geometry plot is wanted
            if show_geometry_plot.value:
                # set text with current year, volume, thickness and velocity
                set_info_text(stage='running model')

                # save new surface/outline and show it on plot
                run_surface_heights[1] = np.array(
                    model.fls[-1].surface_h, ndmin=2)
                glacier_outlines[1] = np.array(
                    np.where(model.fls[-1].thick > 0, widths, nan_array), ndmin=2)
                set_glacier_height_curve()
                set_width_curve()
                set_mb_curve()
                geometry_figure.event(count=geometry_figure_change_index)
                geometry_figure_change_index += 1

            # check if timeseries plot is wanted
            if show_timeseries_plot.value:
                # save volume, length and time of current state
                glacier_volume.append(model.volume_km3)
                glacier_length.append(model.length_m)
                glacier_time.append(glacier_time[-1] + dyears)

                # set volume and length curves with new values
                set_glacier_volume_curve()
                set_glacier_length_curve()

                # force a refresh of the timeseries plot
                timeseries_figure.event(count=timeseries_figure_change_index)
                timeseries_figure_change_index += 1

        # check if geometry plot is wanted
        if show_geometry_plot.value:
            if model.yr >= max_calc_years:
                set_info_text(stage='aborted model')
            else:
                # set text with equilibrium year
                set_info_text(stage='equilibrium')

            geometry_figure.event(count=geometry_figure_change_index)
            geometry_figure_change_index += 1

    elif stage == 'advance years':
        while (model.yr < run_model_years.value):
            model.run_until(model.yr + dyears)

            # check if geometry plot is wanted
            if show_geometry_plot.value:
                # set text with current year and volume
                set_info_text(stage='running model')

                # save new surface/outline and show it on plot
                run_surface_heights[1] = np.array(
                    model.fls[-1].surface_h, ndmin=2)
                glacier_outlines[1] = np.array(
                    np.where(model.fls[-1].thick > 0, widths, nan_array), ndmin=2)
                set_glacier_height_curve()
                set_width_curve()
                set_mb_curve()
                geometry_figure.event(count=geometry_figure_change_index)
                geometry_figure_change_index += 1

            # check if timeseries plot is wanted
            if show_timeseries_plot.value:
                # save volume, length and time of current state
                glacier_volume.append(model.volume_km3)
                glacier_length.append(model.length_m)
                glacier_time.append(glacier_time[-1] + dyears)

                # set volume and length curves with new values
                set_glacier_volume_curve()
                set_glacier_length_curve()

                # force a refresh of the timeseries plot
                timeseries_figure.event(count=timeseries_figure_change_index)
                timeseries_figure_change_index += 1

        # check if geometry plot is wanted
        if show_geometry_plot.value:
            # set text after calculation
            set_info_text(stage='stopped model')
            geometry_figure.event(count=geometry_figure_change_index)
            geometry_figure_change_index += 1

    # check if timeseries plot is wanted
    if show_timeseries_plot.value:
        # store time for vertical lines in volume and length plot
        timeseries_vlin_pos.append(glacier_time[-1])

        # set volume and length curves with new vertical line
        set_glacier_volume_curve()
        set_glacier_length_curve()
        timeseries_figure.event(count=timeseries_figure_change_index)
        timeseries_figure_change_index += 1

        # store values for table
        years_table.append(int(model.yr))
        ELA_table.append(ELA_height.value)
        mb_gradient_table.append(mb_gradient.value)
        glen_a_table.append(glens_creep_parameter.value)
        if sliding_parameter.value == 'sliding':
            fs_table.append('Yes')
        else:
            fs_table.append('No')
        volume_table.append(np.round(model.volume_km3, decimals=3))
        length_table.append(int(np.round(model.length_m)))

        # set the new values in the table
        set_timeseries_table()

        # force the table to refresh
        timeseries_table_DynamicMap.event(count=timeseries_table_change_index)
        timeseries_table_change_index += 1

## Functions for dynamic maps (arrangement of subplots)

This functions define how the subplots will be arranged.

### change geometry figure

In [ ]:
def change_geometry_figure(count=0):
    return ((glacier_height_curve * bed_rock_height_curve).opts(legend_position='bottom_left',
                                                                legend_cols=2,
                                                                xaxis='top',
                                                                bgcolor='lightblue',
                                                                responsive=True,
                                                                aspect=2.5) +
            (mb_curve).opts(legend_position='bottom_right',
                            legend_cols=4,
                            xaxis='top',
                            yaxis='right',
                            bgcolor='lightgray',
                            responsive=True,
                            aspect=2.5) +
            (bed_rock_width_curve * width_curve).opts(responsive=True,
                                                      aspect=6.1,
                                                      yformatter='%8.0f',
                                                      bgcolor='darkgray',
                                                      labelled=['y']) +
            (info_text).opts(responsive=True,
                             aspect=6.1)
            ).opts(sizing_mode='stretch_both',
                   toolbar='right').cols(2)

### change timeseries figure

In [ ]:
def change_timeseries_figure(count=0):
    return ((glacier_volume_curve).opts(responsive=True,
                                        xaxis='top',
                                        aspect=4.7,
                                        max_height=180) +
            (glacier_length_curve).opts(responsive=True,
                                        labelled=['y'],
                                        aspect=5.1,
                                        max_height=120)
            ).opts(sizing_mode='stretch_both',
                   toolbar='left').cols(1)

### change timeseries table

In [ ]:
def change_timeseries_table(count=0):
    return timeseries_table.opts(height=300)

## streamers and dynamic maps

before creating the dynamic maps initialise the model and all subplots with the default values

In [ ]:
init_model()

dynamic map and stream for geometry plot

In [ ]:
geometry_figure_stream = Stream.define('geometry_figure_stream', count=0)

geometry_figure = hv.DynamicMap(
    change_geometry_figure, streams=[geometry_figure_stream()])

dynamic maps and streams for timeseries plot (figure and table)

In [ ]:
timeseries_figure_stream = Stream.define('timeseries_figure_stream', count=0)

timeseries_figure = hv.DynamicMap(
    change_timeseries_figure, streams=[timeseries_figure_stream()])

In [ ]:
timeseries_table_stream = Stream.define('timeseries_table_stream', count=0)

timeseries_table_DynamicMap = hv.DynamicMap(
    change_timeseries_table, streams=[timeseries_table_stream()])

## put figures together in tab menu

In [ ]:
figures = pn.Tabs(('geometry plot', geometry_figure),
                  ('timeseries plot', pn.Row(timeseries_figure,
                                             timeseries_table_DynamicMap,
                                             sizing_mode='stretch_both')),
                  tabs_location='below',
                  sizing_mode='stretch_both')

## App

In [ ]:
pn.Column(header, menu, figures, sizing_mode='stretch_both').show()

As long as you are running this notebook "live" (in Jupyter, not viewing a website or a static copy), the above notebook cell should contain the fully operational dashboard here in the notebook. You can also launch the dashboard at a separate port that shows up in a new browser tab, either by changing .servable() to .show() above and re-executing that cell, or by leaving the cell as it is and running bokeh serve --show simulator.ipynb.